In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

examples = [
{
"movie": "Parasite",
"answer": """
Here is what I know:
Year: 2019
Director: Bong Joon-ho
Starring: Song Kang-ho, Lee Sun-kyun, Cho Yeo-jeong
Budget: $15.5 million
Box office: $262.7 million
Genre: Dark Comedy, Thriller
Synopsis: "Parasite" follows the Kim family, residing in a cramped semi-basement, as they hatch a complex plan for their son Ki-woo to tutor the affluent Park family. However, their scheme to improve their lives by infiltrating the Parks' household ultimately leads to unforeseen and dramatic outcomes.
""",
},
{
"movie": "Wonka",
"answer": """
I know this:
Year: 2023
Director: Paul King
Starring: Timothée Chalamet, Calah Lane, Keegan-Michael Key
Budget: $125 million
Box office: $629.7 million
Genre: Musical, Fantasy
Synopsis: "Wonka" depicts the formative years of Willy Wonka, tracing his journey across Europe to fulfill his dream of opening a chocolate shop. Along the way, he encounters diverse characters, including an orphan who becomes both his ally and competitor, culminating in the creation of his iconic chocolate factory.
""",
},
{
"movie": "Interstellar",
"answer": """
I know this:
Year: 2014
Director: Christopher Nolan
Starring: Matthew McConaughey, Anne Hathaway, Jessica Chastain
Budget: $165 million
Box office: $731 million
Genre: Epic Science Fiction
Synopsis: Set in a future where Earth is ravaged by famine, "Interstellar" follows a group of astronauts traveling through a wormhole near Saturn in search of a new home for humanity​ 
""",
},
{
"movie": "Avatar: The Way of Water",
"answer": """
I know this:
Year: 2022
Director: James Cameron
Starring: Sam Worthington, Zoe Saldaña, and Sigourney Weaver
Budget:  $350–460 million
Box office: $2.32 billion
Genre: Epic Science Fiction
Synopsis: This epic science fiction film, as the second installment in the "Avatar" series, explores Jake Sully's life as he and his family face new threats on Pandora and seek refuge with the aquatic Metkayina clan​ 
""",
},
{
"movie": "Frozen II",
"answer": """
I know this:
Year: 2019
Director: Chris Buck and Jennifer Lee
Starring: Kristen Bell and Idina Menzel
Budget: $150 million
Box office: $1.453 billion
Genre: Musical, Fantasy
Synopsis:  This musical fantasy sequel follows Elsa and Anna on an adventure to discover the origin of Elsa's powers. Despite comparisons to its predecessor, it achieved significant box office success and received positive critiques​
""",
},
]


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {movie}?"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a film critic, you give short answers"),
        example_prompt,
        ("human", "What do you know about {movie}?")
    ]
)

chain = final_prompt | chat

chain.invoke({
    "movie": "Wreck-It Ralph"
})

I know this:
Year: 2012
Director: Rich Moore
Starring: John C. Reilly, Sarah Silverman, Jack McBrayer
Budget: $165 million
Box office: $471.2 million
Genre: Animated, Comedy
Synopsis: "Wreck-It Ralph" follows the titular character, a video game villain who wants to be a hero. He ventures into other games to prove himself, forming an unlikely friendship with a glitch named Vanellope von Schweetz along the way​

AIMessage(content='I know this:\nYear: 2012\nDirector: Rich Moore\nStarring: John C. Reilly, Sarah Silverman, Jack McBrayer\nBudget: $165 million\nBox office: $471.2 million\nGenre: Animated, Comedy\nSynopsis: "Wreck-It Ralph" follows the titular character, a video game villain who wants to be a hero. He ventures into other games to prove himself, forming an unlikely friendship with a glitch named Vanellope von Schweetz along the way\u200b', response_metadata={'finish_reason': 'stop'})